# 0) Clean image performance

In [66]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

files = {
    "internvl": "internvl_clean_baseline.csv",
    "lingshu": "lingshu_clean_baseline.csv",
    "medgemma": "medgemma_clean_baseline.csv",
    "qwen3": "qwen3_clean_baseline.csv",
}

modalities = ["MRI", "OCT", "Xray"]

results = []

for model, path in files.items():
    df = pd.read_csv(path)

    # 타입 정리 (문자열/공백/NaN 방지)
    df["gt_binary"] = pd.to_numeric(df["gt_binary"], errors="coerce")
    df["pred_binary"] = pd.to_numeric(df["pred_binary"], errors="coerce")
    df["is_correct"] = pd.to_numeric(df["is_correct"], errors="coerce")

    for mod in modalities:
        sub = df[df["dataset"] == mod].copy()
        if sub.empty:
            continue

        # label이 NaN인 행 제거
        sub = sub.dropna(subset=["gt_binary", "pred_binary"])
        y_true = sub["gt_binary"].astype(int)
        y_pred = sub["pred_binary"].astype(int)

        results.append({
            "model": model,
            "dataset": mod,
            "n_samples": len(sub),
            "accuracy": accuracy_score(y_true, y_pred),
            "precision": precision_score(y_true, y_pred, zero_division=0),
            "recall": recall_score(y_true, y_pred, zero_division=0),
            "f1": f1_score(y_true, y_pred, zero_division=0),
            # sanity check: is_correct가 제대로면 아래 두 값이 같아야 함
            "is_correct_mean": sub["is_correct"].mean(),
        })

results_df = pd.DataFrame(results)

pivot = (
    results_df
    .pivot_table(
        index=["model", "dataset"],
        values=["n_samples", "accuracy", "precision", "recall", "f1"],
    )
    .sort_index()
)

pivot = pivot[["n_samples", "accuracy", "precision", "recall", "f1"]]
#print(pivot.round(4))

from IPython.display import display

metrics = ["n_samples", "accuracy", "precision", "recall", "f1"]
dataset_order = ["MRI", "OCT", "Xray"]

for model in results_df["model"].unique():
    print(f"\n🤖 {model.upper()} — Clean baseline")

    tbl = (
        results_df[results_df["model"] == model]
        .set_index("dataset")[metrics]
        .reindex(dataset_order)
    )

    display(
        tbl.style
        .format({
            "accuracy":"{:.4f}",
            "precision":"{:.4f}",
            "recall":"{:.4f}",
            "f1":"{:.4f}",
            "n_samples":"{:.0f}",
        })
        .background_gradient(subset=["f1"])
    )



🤖 INTERNVL — Clean baseline


,n_samples,accuracy,precision,recall,f1
dataset,,,,,
MRI,20,0.8000,1.0000,0.6000,0.7500
OCT,20,0.7000,1.0000,0.4000,0.5714
Xray,20,0.7500,0.8571,0.6000,0.7059



🤖 LINGSHU — Clean baseline


,n_samples,accuracy,precision,recall,f1
dataset,,,,,
MRI,20,1.0000,1.0000,1.0000,1.0000
OCT,20,0.7000,1.0000,0.4000,0.5714
Xray,20,0.9000,1.0000,0.8000,0.8889



🤖 MEDGEMMA — Clean baseline


,n_samples,accuracy,precision,recall,f1
dataset,,,,,
MRI,20,0.8500,0.7692,1.0000,0.8696
OCT,20,0.5000,0.0000,0.0000,0.0000
Xray,20,0.8500,1.0000,0.7000,0.8235



🤖 QWEN3 — Clean baseline


,n_samples,accuracy,precision,recall,f1
dataset,,,,,
MRI,20,0.8000,1.0000,0.6000,0.7500
OCT,20,0.7500,0.7273,0.8000,0.7619
Xray,20,0.4000,0.3333,0.2000,0.2500


# 1) Weak image performance

In [75]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from pathlib import Path

models = ["internvl", "lingshu", "medgemma", "qwen3"]

prompts = [
    "baseline-prompts",
    "disease-cue-reason-1-prompts",
    "disease_cue-1-prompts",
    "artefact-disease-cue-2-prompts",
    "artefact_disease_cue-1-prompts",
    "artefact-disease-cue-reason-1-prompts",
]

modalities = ["MRI", "OCT", "Xray"]

results = []

for model in models:
    for prompt in prompts:
        file = f"{model}_{prompt}.csv"
        if not Path(file).exists():
            continue

        df = pd.read_csv(file)

        # weak만
        df = df[df["severity"] == "weak"].copy()

        # 숫자 변환 (문자열/공백/이상값 방지)
        df["binarylabel"] = pd.to_numeric(df["binarylabel"], errors="coerce")
        df["pred_binary"] = pd.to_numeric(df["pred_binary"], errors="coerce")

        for mod in modalities:
            sub = df[df["dataset"] == mod].copy()
            if sub.empty:
                continue

            n_total = len(sub)
            n_gt_nan = int(sub["binarylabel"].isna().sum())
            n_pred_nan = int(sub["pred_binary"].isna().sum())

            # metric 계산 가능한 유효 row만 사용
            valid = sub.dropna(subset=["binarylabel", "pred_binary"]).copy()
            n_valid = len(valid)

            if n_valid == 0:
                results.append({
                    "model": model,
                    "prompt": prompt,
                    "dataset": mod,
                    "n_total": n_total,
                    "n_valid": 0,
                    "n_gt_nan": n_gt_nan,
                    "n_pred_nan": n_pred_nan,
                    "accuracy": None,
                    "precision": None,
                    "recall": None,
                    "f1": None,
                })
                continue

            y_true = valid["binarylabel"].astype(int)
            y_pred = valid["pred_binary"].astype(int)

            results.append({
                "model": model,
                "prompt": prompt,
                "dataset": mod,
                "n_total": n_total,
                "n_valid": n_valid,
                "n_gt_nan": n_gt_nan,
                "n_pred_nan": n_pred_nan,
                "accuracy": accuracy_score(y_true, y_pred),
                "precision": precision_score(y_true, y_pred, zero_division=0),
                "recall": recall_score(y_true, y_pred, zero_division=0),
                "f1": f1_score(y_true, y_pred, zero_division=0),
            })

results_df = pd.DataFrame(results)
pivot = results_df.pivot_table(
    index=["model", "prompt", "dataset"],
    values=["n_valid", "accuracy", "precision", "recall", "f1"],
    aggfunc="first",
).sort_index()

from IPython.display import display

model_order = ["internvl", "lingshu", "medgemma", "qwen3"]
dataset_order = ["MRI", "OCT", "Xray"]
metrics = ["n_valid", "accuracy", "precision", "recall", "f1"]

prompt_order = [
    "baseline-prompts",
    "disease_cue-1-prompts",
    "artefact_disease_cue-1-prompts",
    "artefact-disease-cue-2-prompts",
]


for prompt in prompt_order:
    df_p = results_df[results_df["prompt"] == prompt].copy()
    if df_p.empty:
        continue

    print(f"\n🧾 PROMPT: {prompt}\n" + "─" * 60)

    for model in model_order:
        df_pm = df_p[df_p["model"] == model].copy()
        if df_pm.empty:
            continue

        tbl = (
            df_pm
            .set_index("dataset")[metrics]
            .reindex(dataset_order)
        )

        print(f"\n🤖 Model: {model}")
        display(tbl.round(4))



🧾 PROMPT: baseline-prompts
────────────────────────────────────────────────────────────

🤖 Model: internvl


,n_valid,accuracy,precision,recall,f1
dataset,,,,,
MRI,100,0.77,1.0000,0.54,0.7013
OCT,100,0.55,1.0000,0.10,0.1818
Xray,100,0.63,0.9333,0.28,0.4308



🤖 Model: lingshu


,n_valid,accuracy,precision,recall,f1
dataset,,,,,
MRI,100,0.89,0.9535,0.82,0.8817
OCT,100,0.64,1.0000,0.28,0.4375
Xray,100,0.69,0.6418,0.86,0.7350



🤖 Model: medgemma


...

,n_valid,accuracy,precision,recall,f1
dataset,,,,,
MRI,100,0.85,0.9730,0.72,0.8276
OCT,100,0.59,1.0000,0.18,0.3051
Xray,100,0.71,0.7442,0.64,0.6882



🤖 Model: medgemma


,n_valid,accuracy,precision,recall,f1
dataset,,,,,
MRI,100,0.86,0.8462,0.88,0.8627
OCT,100,0.50,0.0000,0.00,0.0000
Xray,100,0.61,1.0000,0.22,0.3607



🤖 Model: qwen3


,n_valid,accuracy,precision,recall,f1
dataset,,,,,
MRI,100,0.62,1.0,0.24,0.3871
OCT,100,0.50,0.0,0.00,0.0000
Xray,100,0.52,1.0,0.04,0.0769


In [76]:
from IPython.display import display

model_order = ["internvl", "lingshu", "medgemma", "qwen3"]
dataset_order = ["MRI", "OCT", "Xray"]
metrics = ["accuracy", "precision", "recall", "f1"]

prompt_order = [
    "baseline-prompts",
    "disease-cue-reason-1-prompts",
    "disease_cue-1-prompts",
    "artefact-disease-cue-2-prompts",
    "artefact_disease_cue-1-prompts",
    "artefact-disease-cue-reason-1-prompts",
]
for prompt in prompt_order:
    df_p = results_df[results_df["prompt"] == prompt].copy()
    if df_p.empty:
        continue

    print(f"\n🧾 PROMPT: {prompt}")
    print("─" * 70)

    # (model, dataset)를 행으로
    tbl = (
        df_p
        .pivot_table(
            index=["model", "dataset"],
            values=["n_valid"] + metrics,
            aggfunc="first",
        )
        .reindex(
            pd.MultiIndex.from_product(
                [model_order, dataset_order],
                names=["model", "dataset"]
            )
        )
    )

    display(
        tbl.round(4)
        .style
        .background_gradient(subset=["f1"])
        .set_caption(f"Weak performance — Prompt: {prompt}")
    )



🧾 PROMPT: baseline-prompts
──────────────────────────────────────────────────────────────────────



🧾 PROMPT: disease-cue-reason-1-prompts
──────────────────────────────────────────────────────────────────────



🧾 PROMPT: disease_cue-1-prompts
──────────────────────────────────────────────────────────────────────



🧾 PROMPT: artefact-disease-cue-2-prompts
──────────────────────────────────────────────────────────────────────



🧾 PROMPT: artefact_disease_cue-1-prompts
──────────────────────────────────────────────────────────────────────



🧾 PROMPT: artefact-disease-cue-reason-1-prompts
──────────────────────────────────────────────────────────────────────


In [77]:
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from IPython.display import display

# =========================
# 0) 설정
# =========================
models = ["internvl", "lingshu", "medgemma", "qwen3"]

prompts = [
    "baseline-prompts",
    "disease-cue-reason-1-prompts",
    "disease_cue-1-prompts",
    "artefact-disease-cue-2-prompts",
    "artefact_disease_cue-1-prompts",
    "artefact-disease-cue-reason-1-prompts",
]

datasets = ["MRI", "OCT", "Xray"]  # ✅ Xray로 고정
BEST_METRIC = "f1"                # best 기준 (추천: f1)

clean_files = {
    "internvl": "internvl_clean_baseline.csv",
    "lingshu": "lingshu_clean_baseline.csv",
    "medgemma": "medgemma_clean_baseline.csv",
    "qwen3": "qwen3_clean_baseline.csv",
}

def norm_dataset(s: pd.Series) -> pd.Series:
    return s.replace({"XRAY": "Xray", "XRay": "Xray", "xray": "Xray"})

# =========================
# 1) Clean metrics
# =========================
clean_rows = []
for model, path in clean_files.items():
    if not Path(path).exists():
        print(f"⚠️ missing clean file: {path}")
        continue

    df = pd.read_csv(path)
    df["dataset"] = norm_dataset(df["dataset"])
    df["gt_binary"] = pd.to_numeric(df["gt_binary"], errors="coerce")
    df["pred_binary"] = pd.to_numeric(df["pred_binary"], errors="coerce")

    for ds in datasets:
        sub = df[df["dataset"] == ds].copy()
        if sub.empty:
            continue
        valid = sub.dropna(subset=["gt_binary", "pred_binary"]).copy()
        if valid.empty:
            continue

        y_true = valid["gt_binary"].astype(int)
        y_pred = valid["pred_binary"].astype(int)

        clean_rows.append({
            "model": model,
            "dataset": ds,
            "clean_n": len(valid),
            "clean_accuracy": accuracy_score(y_true, y_pred),
            "clean_precision": precision_score(y_true, y_pred, zero_division=0),
            "clean_recall": recall_score(y_true, y_pred, zero_division=0),
            "clean_f1": f1_score(y_true, y_pred, zero_division=0),
        })

clean_df = pd.DataFrame(clean_rows)

# =========================
# 2) Weak metrics (severity=weak)
# =========================
weak_rows = []
for model in models:
    for prompt in prompts:
        file = f"{model}_{prompt}.csv"
        if not Path(file).exists():
            continue

        df = pd.read_csv(file)
        df = df[df["severity"] == "weak"].copy()

        df["dataset"] = norm_dataset(df["dataset"])
        df["binarylabel"] = pd.to_numeric(df["binarylabel"], errors="coerce")
        df["pred_binary"] = pd.to_numeric(df["pred_binary"], errors="coerce")

        for ds in datasets:
            sub = df[df["dataset"] == ds].copy()
            if sub.empty:
                continue

            n_total = len(sub)
            n_gt_nan = int(sub["binarylabel"].isna().sum())
            n_pred_nan = int(sub["pred_binary"].isna().sum())

            valid = sub.dropna(subset=["binarylabel", "pred_binary"]).copy()
            n_valid = len(valid)

            if n_valid == 0:
                weak_rows.append({
                    "model": model, "prompt": prompt, "dataset": ds,
                    "n_total": n_total, "n_valid": 0,
                    "n_gt_nan": n_gt_nan, "n_pred_nan": n_pred_nan,
                    "accuracy": pd.NA, "precision": pd.NA, "recall": pd.NA, "f1": pd.NA,
                })
                continue

            y_true = valid["binarylabel"].astype(int)
            y_pred = valid["pred_binary"].astype(int)

            weak_rows.append({
                "model": model, "prompt": prompt, "dataset": ds,
                "n_total": n_total, "n_valid": n_valid,
                "n_gt_nan": n_gt_nan, "n_pred_nan": n_pred_nan,
                "accuracy": accuracy_score(y_true, y_pred),
                "precision": precision_score(y_true, y_pred, zero_division=0),
                "recall": recall_score(y_true, y_pred, zero_division=0),
                "f1": f1_score(y_true, y_pred, zero_division=0),
            })

weak_df = pd.DataFrame(weak_rows)

# =========================
# 3) Merge + drop
# =========================
merged = weak_df.merge(clean_df, on=["model", "dataset"], how="left")

for m in ["accuracy", "precision", "recall", "f1"]:
    merged[f"{m}_drop"] = merged[m] - merged[f"clean_{m}"]

# =========================
# 4) Best prompt (model × dataset)
# =========================
best_df = (
    merged.dropna(subset=[BEST_METRIC])
    .sort_values([BEST_METRIC, "n_valid"], ascending=[False, False])
    .groupby(["model", "dataset"], as_index=False)
    .first()
)

# =========================
# 5) 예쁘게 출력 🎨
# =========================

# (A) Clean 요약 (짧게)
print("🧼 CLEAN baseline summary")
clean_pivot = (
    clean_df.pivot_table(index="model", columns="dataset", values="clean_f1", aggfunc="first")
    .reindex(models)
    .reindex(columns=datasets)
)
display(clean_pivot.style.format("{:.4f}").set_caption("Clean F1 (higher is better)"))

# (B) BEST prompt 요약표 (가장 중요 🏆)
print("\n🏆 BEST prompt per (model × modality) — based on F1 (weak only)")
best_show = best_df[[
    "model","dataset","prompt","n_valid",
    "f1","clean_f1","f1_drop",
    "recall","precision","accuracy"
]].copy()

# 보기 좋게 모델/모달리티 순서 고정
best_show["model"] = pd.Categorical(best_show["model"], categories=models, ordered=True)
best_show["dataset"] = pd.Categorical(best_show["dataset"], categories=datasets, ordered=True)
best_show = best_show.sort_values(["model","dataset"])

display(
    best_show.style
    .format({
        "f1":"{:.4f}", "clean_f1":"{:.4f}", "f1_drop":"{:+.4f}",
        "recall":"{:.4f}", "precision":"{:.4f}", "accuracy":"{:.4f}"
    })
    .background_gradient(subset=["f1"], axis=None)                 # f1 높을수록 강조
    .background_gradient(subset=["f1_drop"], axis=None)            # drop도 시각화
    .set_caption("Best prompt (weak) + clean reference + drop")
)

# (C) 모델별로 짧게: 모달리티 3줄짜리 best만 보여주기 🤖
print("\n🤖 Model-wise best prompts (compact view)")
for model in models:
    df_m = best_show[best_show["model"] == model].copy()
    if df_m.empty:
        continue

    compact = df_m.set_index("dataset")[["prompt","n_valid","f1","f1_drop","clean_f1","recall","precision","accuracy"]]
    compact = compact.reindex(datasets)

    print(f"\n✨ {model.upper()}")
    display(
        compact.style
        .format({
            "f1":"{:.4f}", "clean_f1":"{:.4f}", "f1_drop":"{:+.4f}",
            "recall":"{:.4f}", "precision":"{:.4f}", "accuracy":"{:.4f}"
        })
        .background_gradient(subset=["f1"], axis=None)
        .background_gradient(subset=["f1_drop"], axis=None)
    )


🧼 CLEAN baseline summary


dataset,MRI,OCT,Xray
model,,,
internvl,0.7500,0.5714,0.7059
lingshu,1.0000,0.5714,0.8889
medgemma,0.8696,0.0000,0.8235
qwen3,0.7500,0.7619,0.2500



🏆 BEST prompt per (model × modality) — based on F1 (weak only)


,model,dataset,prompt,n_valid,f1,clean_f1,f1_drop,recall,precision,accuracy
0,internvl,MRI,baseline-prompts,100,0.7013,0.7500,-0.0487,0.5400,1.0000,0.7700
1,internvl,OCT,baseline-prompts,100,0.1818,0.5714,-0.3896,0.1000,1.0000,0.5500
2,internvl,Xray,disease_cue-1-prompts,100,0.5217,0.7059,-0.1841,0.3600,0.9474,0.6700
3,lingshu,MRI,baseline-prompts,100,0.8817,1.0000,-0.1183,0.8200,0.9535,0.8900
4,lingshu,OCT,baseline-prompts,100,0.4375,0.5714,-0.1339,0.2800,1.0000,0.6400
5,lingshu,Xray,baseline-prompts,100,0.7350,0.8889,-0.1538,0.8600,0.6418,0.6900
6,medgemma,MRI,disease_cue-1-prompts,100,0.8627,0.8696,-0.0068,0.8800,0.8462,0.8600
7,medgemma,OCT,disease_cue-1-prompts,100,0.0392,0.0000,+0.0392,0.0200,1.0000,0.5100
8,medgemma,Xray,disease_cue-1-prompts,100,0.3871,0.8235,-0.4364,0.2400,1.0000,0.6200
9,qwen3,MRI,disease_cue-1-prompts,100,0.5294,0.7500,-0.2206,0.3600,1.0000,0.6800



🤖 Model-wise best prompts (compact view)

✨ INTERNVL


...

,prompt,n_valid,f1,f1_drop,clean_f1,recall,precision,accuracy
dataset,,,,,,,,
MRI,baseline-prompts,100,0.8817,-0.1183,1.0000,0.8200,0.9535,0.8900
OCT,baseline-prompts,100,0.4375,-0.1339,0.5714,0.2800,1.0000,0.6400
Xray,baseline-prompts,100,0.7350,-0.1538,0.8889,0.8600,0.6418,0.6900



✨ MEDGEMMA


,prompt,n_valid,f1,f1_drop,clean_f1,recall,precision,accuracy
dataset,,,,,,,,
MRI,disease_cue-1-prompts,100,0.8627,-0.0068,0.8696,0.8800,0.8462,0.8600
OCT,disease_cue-1-prompts,100,0.0392,+0.0392,0.0000,0.0200,1.0000,0.5100
Xray,disease_cue-1-prompts,100,0.3871,-0.4364,0.8235,0.2400,1.0000,0.6200



✨ QWEN3


,prompt,n_valid,f1,f1_drop,clean_f1,recall,precision,accuracy
dataset,,,,,,,,
MRI,disease_cue-1-prompts,100,0.5294,-0.2206,0.7500,0.3600,1.0000,0.6800
OCT,baseline-prompts,100,0.2069,-0.5550,0.7619,0.1200,0.7500,0.5400
Xray,disease_cue-1-prompts,100,0.2456,-0.0044,0.2500,0.1400,1.0000,0.5700


# 3) Strong image performance

In [78]:
import pandas as pd
from pathlib import Path

models = ["internvl", "lingshu", "medgemma", "qwen3"]

datasets = ["MRI", "OCT", "Xray"]

rows = []

for model in models:
    for prompt in prompts:
        file = f"{model}_{prompt}.csv"
        if not Path(file).exists():
            continue

        df = pd.read_csv(file)

        # 🔴 strong만
        df = df[df["severity"] == "strong"].copy()
        if df.empty:
            continue

        # dataset 표기 통일
        df["dataset"] = df["dataset"].replace({"XRAY": "Xray", "XRay": "Xray"})

        for ds in datasets:
            sub = df[df["dataset"] == ds].copy()
            if sub.empty:
                continue

            n_total = len(sub)
            n_distorted = (sub["model_pred"] == "distorted").sum()

            rows.append({
                "model": model,
                "prompt": prompt,
                "dataset": ds,
                "n_total": n_total,
                "n_distorted": int(n_distorted),
                "distorted_ratio": n_distorted / n_total,
            })

strong_df = pd.DataFrame(rows)
#print(strong_df.round(4))


In [79]:
from IPython.display import display

for prompt in strong_df["prompt"].unique():
    print(f"\n🧨 STRONG — PROMPT: {prompt}")

    tbl = (
        strong_df[strong_df["prompt"] == prompt]
        .pivot_table(
            index="model",
            columns="dataset",
            values="distorted_ratio",
            aggfunc="first",
        )
        .reindex(index=models, columns=datasets)
    )

    display(
        tbl.style
        .format("{:.2%}")
        .background_gradient(axis=None)
        .set_caption("Distorted output ratio (severity = strong)")
    )



🧨 STRONG — PROMPT: baseline-prompts


dataset,MRI,OCT,Xray
model,,,
internvl,14.00%,8.00%,44.00%
lingshu,11.00%,2.00%,35.00%
medgemma,1.00%,0.00%,1.00%
qwen3,2.00%,1.00%,14.00%



🧨 STRONG — PROMPT: disease-cue-reason-1-prompts


dataset,MRI,OCT,Xray
model,,,
internvl,0.00%,0.00%,0.00%
lingshu,16.00%,1.00%,43.00%
medgemma,0.00%,0.00%,0.00%
qwen3,2.00%,0.00%,6.00%



🧨 STRONG — PROMPT: disease_cue-1-prompts


...

dataset,MRI,OCT,Xray
model,,,
internvl,2.00%,5.00%,0.00%
lingshu,17.00%,4.00%,52.00%
medgemma,4.00%,4.00%,4.00%
qwen3,3.00%,2.00%,10.00%



🧨 STRONG — PROMPT: artefact_disease_cue-1-prompts


dataset,MRI,OCT,Xray
model,,,
internvl,0.00%,3.00%,0.00%
lingshu,8.00%,1.00%,41.00%
medgemma,2.00%,2.00%,2.00%
qwen3,0.00%,0.00%,0.00%



🧨 STRONG — PROMPT: artefact-disease-cue-reason-1-prompts


dataset,MRI,OCT,Xray
model,,,
internvl,0.00%,0.00%,0.00%
lingshu,26.00%,3.00%,53.00%
medgemma,2.00%,0.00%,1.00%
qwen3,3.00%,1.00%,7.00%
